In [1]:
import os
import re
import pandas as pd
from more_itertools import locate

In [2]:
Paths = {
    'Data': './../Data/',
    'Chia_w_scope': './../Data/chia_with_scope/',
    'Chia_wo_scope': './../Data/chia_without_scope/',
}


entity_types = ['Condition', 'Drug', 'Procedure', 'Measurement', 'Observation', 'Person', 'Device', \
    'Value', 'Temporal', 'Qualifier', 'Negation']

relation_type = ['OR', 'AND', 'Has_qualifier', 'Has_value', 'Has_negation', 'Has_temporal', 'Has_context']

In [7]:
os.listdir(Paths['Chia_wo_scope'])[:5]

['NCT00599924_inc.ann',
 'NCT02974686_inc.ann',
 'NCT03209687_exc.ann',
 'NCT03100513_exc.txt',
 'NCT02053246_exc.txt']

In [8]:
def get_Input_Files(file_name):

    inputfiles = set()
    for f in os.listdir(Paths[file_name]):
        if f.endswith('.ann'):
            inputfiles.add(f.split('.')[0].split('_')[0])
    input_files = list(inputfiles)
    print(len(inputfiles))
#     return inputfiles

get_Input_Files('Chia_w_scope')
get_Input_Files('Chia_wo_scope')

1000
1000


In [9]:
class Data_Preparation:
    def __init__(self, path, relation_type, entity_types):
        self.path = path
        self.entity_types = entity_types
        self.relation_type = relation_type
        self.ignore_files = []
        self.input_files = []
        self.globalEntities = set()
        
        print('Data_Preparation Initialized...')
        
    ###################################################################### HELPER FUNCTIONS
        
    def checkEntityValue(self, e):
        """
        Helper Function for get_annotation_relations
        """
        if e.startswith('T'):
            return e.strip()
        else:
            return e.split(':')[-1].strip()
        
    
    def removePunctuation(self, word):
        """
        Helper Function for get_Entity_Tags
        """
        word = re.sub(r'^(\.|,|\(|\))', '', word)
        word = re.sub(r'(\.|,|\(|\))$', '', word)
        return word
    
    def cleanEntityTokens(self, txt):
        """
        Clean the tokens from / and - for words in entities
        """
        txt = txt.replace('/',' ')
        txt = txt.replace('-',' ')
        txt = txt.replace(';',' ')
        txt = re.sub(r' {2,}', ' ',txt)
        txt = txt.strip()
        
        return txt.split()
    
    def readTxt(self, txt_file):
        with open(txt_file, "r", encoding="utf-8") as f:
            text_array = f.read()
        print(text_array)
        
    def readAnn(self, ann_file):
        with open(ann_file, "r", encoding="utf-8") as f:
            text_array = f.read()
        print(text_array)
        
    def printFiles(self, file_name, file):
        """
        Helper function to print files without manual viewing them
        """
        ann_file = f"{Paths[file_name]}{file}.ann"
        txt_file = f"{Paths[file_name]}{file}.txt"
        self.readAnn(ann_file)
        self.readTxt(txt_file)
    
    
    ######################################################################
    
    def getInputFiles(self, file_name):
        """
        file_name: 'Chia_w_scope' or 'Chia_wo_scope'
        """
        inputfiles = set()
        for f in os.listdir(Paths[file_name]):
            if f.endswith('.ann'):
                inputfiles.add(f.split('.')[0].split('_')[0])
        self.input_files = list(inputfiles)
        return inputfiles

    def ignoreFiles(self, text_array):
        match = re.findall(r'^( {1,}\n$|NA {0,}\n$)',text_array[0])
        if len(match):
            return True
        else:
            return False
    
    def get_annotation_entities(self, ann_file):
        """
        Get all entities which correspond to the entities mentioned in the entity types.
        """
        entities = []
        with open(ann_file, "r", encoding="utf-8") as f:
            for line in f:
                if line.startswith('T'):
                    assert len(line.strip().split('\t')) == 3
                    entity_identity = line.strip().split('\t')[0]
                    entity_token = line.strip().split('\t')[-1]
                    
                    if ';' in line.strip().split('\t')[1]:
                        line = line.replace(';',' ')
                        term = line.strip().split('\t')[1].split()
                        term[-1] = str(max([int(v) for v in term[1:]]))
                        term[1] = str(min([int(v) for v in term[1:]]))
                    else:
                        term = line.strip().split('\t')[1].split()
                        
                    self.globalEntities.add(term[0])
                    
                    if (self.entity_types != None) and (term[0] not in self.entity_types): continue
                    if int(term[-1]) <= int(term[1]):
                        raise RuntimeError('Starting and Ending Indices are off.')
                    entities.append((entity_identity, int(term[1]), int(term[-1]), term[0], entity_token))
                    
        return sorted(entities, key=lambda x: (x[2]))
    
    def remove_overlap_entities(self, sorted_entities):
        """
        If you want to get the largest overlap of entity so two words don't have different entities
        
        Here we just use the uncommented part to get the unique entities which we are considering.
        """
#         keep_entities = []
#         for idx, entity in enumerate(sorted_entities):
#             if idx == 0:
#                 keep_entities.append(entity)
#                 last_keep = entity
#                 continue
#             if entity[1] < last_keep[2]:
#                 if entity[2]-entity[1] > last_keep[2]-last_keep[1]:
#                     last_keep = entity
#                     keep_entities[-1] = last_keep
#             elif entity[1] == last_keep[2]:
#                 last_keep = (last_keep[1], entity[2], last_keep[-1])
#                 keep_entities[-1] = last_keep
#             else:
#                 last_keep = entity
#                 keep_entities.append(entity)

        keep_entities = sorted_entities

        uniqueEntity = []        
        for ent in keep_entities:
            uniqueEntity.append(ent[0])

        return keep_entities, uniqueEntity


    # https://datagy.io/python-list-find-all-index/
    def get_annotation_relations(self, ann_file, uniqueEntity):
        """
        Gives all relations corresponding to the relations mentioned in relations_type
        And make sure they are relations between entities that are mentioned in entity_types
        """
        relations = []
        with open(ann_file, "r", encoding="utf-8") as f:
            for line in f:
                if line.startswith('R') or line.startswith('*'):
                    assert len(line.strip().split('\t')) == 2 # ['R1', 'Has_qualifier Arg1:T6 Arg2:T5']
                    if line.strip().split('\t')[1].split()[0] not in relation_type: continue

                    rel = line.strip().split('\t')[0]
                    rel_type = line.strip().split('\t')[1].split()[0]
                    entities = line.strip().split('\t')[1].split()[1:]
                    entities= [self.checkEntityValue(e) for e in entities]
                    entities = [e for e in entities if e in uniqueEntity]
                    entities = ' '.join(entities)
                    match = re.findall(r'^T[0-9]+ T[0-9]+$',entities)
                    if len(match):
                        pass
                    else:
                        continue
                    assert len(entities.split()) == 2
                    relations.append((rel, rel_type, entities))

        return relations
    
    
    def get_text(self, txt_file, file):
        """
        Get raw text
        """
        with open(txt_file, "r", encoding="utf-8") as f:
            text_array = f.readlines()
            if file in ['NCT02348918_exc', 'NCT02348918_inc', 'NCT01735955_exc']: # Inconsistent offsets (Line breaks)
                text = ' '.join([i.strip() for i in text_array])
            else:
                text = '  '.join([i.strip() for i in text_array])
        
        return text, text_array
    
    
    def get_text_array(self,text_array):
        """
        Get cleaned text in array form
        """
        
        globalText = []
        offset = 0
        for txt in text_array:
            textlen = len(txt)
            
            txt = txt.replace('/',' ')
            txt = txt.replace('-',' ')
            txt = txt.replace(';',' ')
            txt = re.sub(r' {2,}', ' ',txt)
            txt = txt.replace('.\n','')
            txt = txt.replace('\n', '')
            txt = txt.strip()

            globalText.append(([self.removePunctuation(w) for w in txt.split()], offset, offset + textlen)) 
            offset += textlen

        return globalText
    
    def get_NER_Tags(self, globalText, keep_entities):
        """
        Get NER Tags for each token in a sentence.
        Then also return a list of the entity identity e.g. T1, T2 etc.
        """
        WORDS, TAGS, IDENTITY = [], [], []
        offset = 10
        for text in globalText:
            words = text[0]
            tags = ['O']*len(words)
            entity_identity = ['X']*len(words)
            sent_indices = set()
            for k in keep_entities:
                # Using 10 as an offset for wrong offset entries in inc/exc files
                # Only look at entities if start and stop indices with an offset match
                if k[1] >= (text[1]-offset) and k[2] <= (text[2]+offset): 
                    clean_tokens = self.cleanEntityTokens(k[-1])
                    break_down = [self.removePunctuation(v) for v in clean_tokens] # Get all the tokens (punc removed) from entities
                    main_index = 0
                    label = ''
                    for i, w in enumerate(break_down): # Go over entity tokens
                        indices = list(locate(words, lambda x: x == w)) # See if tokens is in sentence

                        if i == 0:
                            try:
                                main_index = indices[0]
                                if len(break_down) > 1: label = 'B-'
                            except:
                                if len(w) <= offset:
                                    pass
                                else:
                                    pass
#                                     raise RuntimeError('Word Length greater than offset')
                        else:
                            label = 'I-'
                        indices= list(filter(lambda x: x >= main_index, indices))
                        indices= list(filter(lambda x: x not in sent_indices, indices))
                        if len(indices) != 0:
                            sent_indices.add(indices[0])
                            tags[indices[0]] = label + k[3]
                            entity_identity[indices[0]] = k[0]
            assert len(words) == len(tags) == len(entity_identity)
            WORDS.append(words)
            TAGS.append(tags) 
            IDENTITY.append(entity_identity)
        return WORDS, TAGS, IDENTITY
    
    def save_to_df(self, FILES, CRITERIA, TEXT, GROUP_ENTITIES, RELATIONS, TOKENS, ENTITIES):
        
        df = pd.DataFrame()

        df['File'] = pd.Series(FILES) 
        df['Criteria'] = pd.Series(CRITERIA) 
        df['Text'] = pd.Series(TEXT)
        df['Group_Entities'] = pd.Series(GROUP_ENTITIES)
        df['Relations'] = pd.Series(RELATIONS)
        df['Tokens'] = pd.Series(TOKENS)
        df['Entities'] = pd.Series(ENTITIES) # (Entity Name, Entity Identity)
        
        return df
    
    def run(self, file_name):
        """
        Run on all files
        """
        inputfiles = self.getInputFiles(file_name)
        
        FILES, CRITERIA, TEXT, GROUP_ENTITIES, RELATIONS, TOKENS, ENTITIES = [], [], [], [], [], [], []
        for infile in inputfiles:
            for t in ["inc", "exc"]:
                file = f"{infile}_{t}"
                ann_file = f"{Paths[file_name]}{file}.ann"
                txt_file = f"{Paths[file_name]}{file}.txt"
                
                text, text_array = self.get_text(txt_file, file)
                ignore = self.ignoreFiles(text_array)
                if ignore: 
                    self.ignore_files.append(file)
                    continue
                sorted_entities = self.get_annotation_entities(ann_file)
                entities, uniqueEntity = self.remove_overlap_entities(sorted_entities)
                relations = self.get_annotation_relations(ann_file, uniqueEntity)
                global_text_array = self.get_text_array(text_array)
                words, tags, entity_identity = self.get_NER_Tags(global_text_array, entities)           
                
                FILES.append(file)
                CRITERIA.append(t)
                TEXT.append(text_array)
                GROUP_ENTITIES.append(entities)
                RELATIONS.append(relations)
                TOKENS.append(words)
                ENTITIES.append((tags, entity_identity))
#             break
                           
        df = self.save_to_df(FILES, CRITERIA, TEXT, GROUP_ENTITIES, RELATIONS, TOKENS, ENTITIES)       
        return df
                
                
    def main(self,files_name):
        """
        Give list of files and from the files extract entities, tokens and relations and save in dataframe format
        """
        
        df = self.run(files_name)
        return df


In [10]:
dataPrep = Data_Preparation(Paths, relation_type, entity_types)

Data_Preparation Initialized...


In [11]:
files = ['Chia_w_scope', 'Chia_wo_scope']
df = dataPrep.main(files[0])

In [13]:
print('Chia_w_scope: ',len(df))
print('Chia_wo_scope: ',len(dataPrep.main(files[1])))

Chia_w_scope:  1950
Chia_wo_scope:  1950


In [14]:
df.head()

File Criteria  \
0  NCT02152696_inc      inc   
1  NCT02152696_exc      exc   
2  NCT02476461_inc      inc   
3  NCT02476461_exc      exc   
4  NCT03226080_inc      inc   

                                                Text  \
0  [Female with a persisting pregnancy of unknown...   
1  [Hemodynamically unstable in need of acute tre...   
2  [symptomatic Dupuytrens contracture with palpa...   
3  [previous treated dupuytrens contracture same ...   
4  [ASA I-IV Age 55 or older Scheduled for operat...   

                                      Group_Entities  \
0  [(T1, 0, 6, Person, Female), (T2, 25, 34, Cond...   
1  [(T1, 0, 24, Condition, Hemodynamically unstab...   
2  [(T2, 0, 11, Qualifier, symptomatic), (T1, 12,...   
3  [(T1, 0, 8, Temporal, previous), (T2, 9, 16, Q...   
4  [(T1, 0, 3, Measurement, ASA), (T2, 4, 8, Valu...   

                                           Relations  \
0  [(R1, Has_qualifier, T2 T3), (R2, Has_value, T...   
1  [(R1, Has_value, T2 T4), (R2, Has_temporal, T2...   
2  [(R1, Has_value, T5 T6), (R2, Has_qualifier, T...   
3  [(R1, Has_qualifier, T3 T4), (R2, Has_qualifie...   
4  [(R1, Has_value, T3 T4), (R2, Has_qualifier, T...   

                                              Tokens  \
0  [[Female, with, a, persisting, pregnancy, of, ...   
1  [[Hemodynamically, unstable, in, need, of, acu...   
2  [[symptomatic, Dupuytrens, contracture, with, ...   
3  [[previous, treated, dupuytrens, contracture, ...   
4  [[ASA, I, IV, Age, 55, or, older, Scheduled, f...   

                                            Entities  
0  ([[Person, O, O, O, Condition, O, B-Qualifier,...  
1  ([[B-Condition, I-Condition, O, O, O, O, O], [...  
2  ([[Qualifier, B-Condition, I-Condition, O, B-C...  
3  ([[Temporal, Qualifier, B-Condition, I-Conditi...  
4  ([[Measurement, B-Value, I-Value, Person, B-Va...

In [15]:
dataPrep.main(files[1]).head()

File Criteria  \
0  NCT02152696_inc      inc   
1  NCT02152696_exc      exc   
2  NCT02476461_inc      inc   
3  NCT02476461_exc      exc   
4  NCT03226080_inc      inc   

                                                Text  \
0  [Female with a persisting pregnancy of unknown...   
1  [Hemodynamically unstable in need of acute tre...   
2  [symptomatic Dupuytrens contracture with palpa...   
3  [previous treated dupuytrens contracture same ...   
4  [ASA I-IV Age 55 or older Scheduled for operat...   

                                      Group_Entities  \
0  [(T1, 0, 6, Person, Female), (T2, 25, 34, Cond...   
1  [(T1, 0, 24, Condition, Hemodynamically unstab...   
2  [(T2, 0, 11, Qualifier, symptomatic), (T1, 12,...   
3  [(T1, 0, 8, Temporal, previous), (T2, 9, 16, Q...   
4  [(T1, 0, 3, Measurement, ASA), (T2, 4, 8, Valu...   

                                           Relations  \
0  [(R1, Has_qualifier, T2 T3), (R2, Has_value, T...   
1  [(R1, Has_value, T2 T4), (R2, Has_temporal, T2...   
2  [(R1, Has_value, T5 T6), (R2, Has_qualifier, T...   
3  [(R1, Has_qualifier, T3 T4), (R2, Has_qualifie...   
4  [(R1, Has_value, T3 T4), (R2, Has_qualifier, T...   

                                              Tokens  \
0  [[Female, with, a, persisting, pregnancy, of, ...   
1  [[Hemodynamically, unstable, in, need, of, acu...   
2  [[symptomatic, Dupuytrens, contracture, with, ...   
3  [[previous, treated, dupuytrens, contracture, ...   
4  [[ASA, I, IV, Age, 55, or, older, Scheduled, f...   

                                            Entities  
0  ([[Person, O, O, O, Condition, O, B-Qualifier,...  
1  ([[B-Condition, I-Condition, O, O, O, O, O], [...  
2  ([[Qualifier, B-Condition, I-Condition, O, B-C...  
3  ([[Temporal, Qualifier, B-Condition, I-Conditi...  
4  ([[Measurement, B-Value, I-Value, Person, B-Va...

In [6]:
dataPrep.ignore_files

['NCT03106389_inc',
 'NCT03106389_exc',
 'NCT03198910_exc',
 'NCT03082573_exc',
 'NCT01088750_exc',
 'NCT02607163_exc',
 'NCT02748330_exc',
 'NCT03389061_exc',
 'NCT02504203_exc',
 'NCT02620904_exc',
 'NCT02256943_exc',
 'NCT03134196_inc',
 'NCT03134196_exc',
 'NCT02590653_exc',
 'NCT03045562_exc',
 'NCT02953873_exc',
 'NCT03615508_exc',
 'NCT03263481_inc',
 'NCT03263481_exc',
 'NCT02969876_exc',
 'NCT03350815_exc',
 'NCT03156855_exc',
 'NCT02918409_exc',
 'NCT01490034_exc',
 'NCT03372304_exc',
 'NCT01352598_exc',
 'NCT02219880_exc',
 'NCT02222272_exc',
 'NCT01742117_exc',
 'NCT02399033_exc',
 'NCT03088904_inc',
 'NCT03088904_exc',
 'NCT03077204_exc',
 'NCT02256956_exc',
 'NCT03011177_exc',
 'NCT02785549_exc',
 'NCT03620526_exc',
 'NCT02715518_exc',
 'NCT02457442_exc',
 'NCT03278548_exc',
 'NCT02478346_exc',
 'NCT03355157_exc',
 'NCT02323399_exc',
 'NCT02944604_exc',
 'NCT02270970_exc',
 'NCT01078051_exc',
 'NCT02408120_exc',
 'NCT01944800_exc',
 'NCT03228238_exc',
 'NCT02042287_exc']

In [7]:
dataPrep.globalEntities

{'Competing_trial',
 'Condition',
 'Context_Error',
 'Device',
 'Drug',
 'Grammar_Error',
 'Informed_consent',
 'Intoxication_considerations',
 'Line',
 'Measurement',
 'Mood',
 'Multiplier',
 'Negation',
 'Non-query-able',
 'Non-representable',
 'Not_a_criteria',
 'Observation',
 'Parsing_Error',
 'Person',
 'Post-eligibility',
 'Pregnancy_considerations',
 'Procedure',
 'Qualifier',
 'Reference_point',
 'Scope',
 'Subjective_judgement',
 'Temporal',
 'Undefined_semantics',
 'Value',
 'Visit',
 'c-Requires_causality'}

In [8]:
dataPrep.printFiles('Chia_w_scope', 'NCT01346436_exc')

T1	Person 0 3	Age
T2	Value 4 17	<18 years old
R1	Has_value Arg1:T1 Arg2:T2	
T3	Post-eligibility 19 75	Patient unable to communicate or to understand the study
T4	Post-eligibility 77 121	Patient refusing to participate to the study
T5	Condition 123 139	contraindication
T6	Procedure 143 154	laparoscopy
R2	AND Arg1:T5 Arg2:T6	

Age <18 years old
Patient unable to communicate or to understand the study
Patient refusing to participate to the study
contraindication to laparoscopy



In [9]:
dataPrep.printFiles('Chia_w_scope', 'NCT03615508_exc')

T1	Non-query-able 0 2	NA

NA




In [10]:
df

File Criteria  \
0     NCT01815580_inc      inc   
1     NCT01815580_exc      exc   
2     NCT03195153_inc      inc   
3     NCT03195153_exc      exc   
4     NCT03115151_inc      inc   
...               ...      ...   
1945  NCT01824537_exc      exc   
1946  NCT02498483_inc      inc   
1947  NCT02498483_exc      exc   
1948  NCT03506750_inc      inc   
1949  NCT03506750_exc      exc   

                                                   Text  \
0     [Adult men who have sex with men, and transgen...   
1     [Prior receipt of investigational anti-HIV vac...   
2     [diabetic patient;\n, therapy with aspirin and...   
3     [not diabetic patient;\n, patients in dual ant...   
4     [Adult subjects aged 18 years or older\n, Sche...   
...                                                 ...   
1945  [Volunteers must not have been vaccinated agai...   
1946  [Apgar score at 5 minutes >7\n, birthweight gr...   
1947  [Newborns of substance abusing mothers.\n, New...   
1948  [18 years or older\n, Type 1 or 2 diabetes\n, ...   
1949  [previous retinal vein occlusion.\n, any intra...   

                                         Group_Entities  \
0     [(T1, 0, 5, Person, Adult), (T2, 6, 31, Person...   
1     [(T1, 0, 5, Temporal, Prior), (T2, 17, 32, Qua...   
2     [(T1, 0, 8, Condition, diabetic), (T2, 32, 39,...   
3     [(T2, 0, 3, Negation, not), (T1, 4, 12, Condit...   
4     [(T1, 0, 5, Person, Adult), (T2, 15, 19, Perso...   
...                                                 ...   
1945  [(T1, 16, 19, Negation, not), (T2, 20, 29, Tem...   
1946  [(T1, 0, 11, Measurement, Apgar score), (T3, 1...   
1947  [(T1, 0, 8, Person, Newborns), (T3, 12, 29, Co...   
1948  [(T1, 0, 17, Value, 18 years or older), (T2, 1...   
1949  [(T2, 0, 8, Temporal, previous), (T1, 9, 31, C...   

                                              Relations  \
0     [(*, OR, T2 T3), (R1, Has_value, T4 T5), (R4, ...   
1     [(R1, Has_qualifier, T3 T2), (R2, Has_temporal...   
2                                      [(*, OR, T2 T3)]   
3     [(R1, Has_negation, T1 T2), (R2, Has_qualifier...   
4     [(R1, Has_value, T2 T3), (R2, AND, T1 T2), (R3...   
...                                                 ...   
1945  [(R1, AND, T3 T4), (R2, Has_negation, T3 T1), ...   
1946  [(R1, Has_value, T1 T2), (R2, Has_temporal, T1...   
1947               [(R1, AND, T2 T3), (R2, AND, T5 T6)]   
1948  [(R1, Has_value, T2 T1), (*, OR, T3 T4), (R3, ...   
1949  [(R1, Has_temporal, T1 T2), (R2, Has_temporal,...   

                                                 Tokens  \
0     [[Adult, men, who, have, sex, with, men, and, ...   
1     [[Prior, receipt, of, investigational, anti, H...   
2     [[diabetic, patient], [therapy, with, aspirin,...   
3     [[not, diabetic, patient], [patients, in, dual...   
4     [[Adult, subjects, aged, 18, years, or, older]...   
...                                                 ...   
1945  [[Volunteers, must, not, have, been, vaccinate...   
1946  [[Apgar, score, at, 5, minutes, >7], [birthwei...   
1947  [[Newborns, of, substance, abusing, mothers], ...   
1948  [[18, years, or, older], [Type, 1, or, 2, diab...   
1949  [[previous, retinal, vein, occlusion], [any, i...   

                                               Entities  
0     ([[Person, B-Person, I-Person, I-Person, I-Per...  
1     ([[Temporal, O, O, Qualifier, B-Drug, I-Drug, ...  
2     ([[Condition, O], [O, O, Drug, O, Drug], [O, B...  
3     ([[Negation, Condition, O], [O, O, B-Procedure...  
4     ([[Person, O, Person, B-Value, I-Value, I-Valu...  
...                                                 ...  
1945  ([[O, O, Negation, B-Temporal, I-Temporal, Pro...  
1946  ([[B-Measurement, I-Measurement, B-Temporal, I...  
1947  ([[Person, O, B-Condition, I-Condition, Person...  
1948  ([[B-Value, I-Value, I-Value, I-Value], [B-Con...  
1949  ([[Temporal, B-Condition, I-Condition, I-Condi...  

[1950 rows x 7 columns]

In [12]:
# df.to_csv('main_data.csv')
# xdf = pd.read_csv('main_data.csv')
# xdf

In [ ]:
xdf.loc[0]

In [ ]:
xdf.loc[0]['Entities']

In [ ]:
for i, f in xdf.iterrows():
    print(f['Tokens'])
    break

In [ ]:
for i, f in xdf.iterrows():
    print(eval(f['Entities'])[0])
    print(eval(f['Entities'])[1])
    break